In [436]:
import tweepy
#from tweepy.parsers import JSONParser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import json
from collections import Counter
import re

In [9]:
consumer_token = ''
consumer_secret = ''

access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [183]:
trump = api.search('Trump')
for tweet in trump:
    print(tweet.text)

Florida residents want to know why Trump isn’t opening his resorts to hurricane victims https://t.co/H3VGXu05dS Gre… https://t.co/SvTzB02b79
RT @TheRynheart: Trump tells cabinet Hurricane Irma means wealthy need their tax cuts faster. 😮
@realDonaldTrump are you kidding me? 
https…
RT @Unpersuaded112: The rich get richer and donald ( @realDonaldTrump ) scews the rest of us #trump #trumpTaxPlan https://t.co/PMKQWG0Tsa
@ajulysantos Tem que rir mesmo. E o Catrina que acabou com New Orleans. Está fixação do Trump e doentia.
RT @SteveBannen: Trump's response to Hillary Clinton's newly-released book:

"You mad, bro?"
RT @todoalnatural: A ver si después de esto, Donald Trump sigue diciendo que el 'cambio climático' es puro cuento. https://t.co/jEia7Fz6LA
RT @tedlieu: Trump Jr. testimony confirms the initial public statement about the Russia meeting by @DonaldJTrumpJr &amp; @realDonaldTrump was a…
Trump to go to Florida 'very soon': https://t.co/PWn8baS6N8 via @YouTube
RT @ProgressOutlook: Robe

In [44]:
trump_2 = api.user_timeline(screen_name = 'realDonaldTrump', count = 200)

#What kind of stuff can we do with this object? (tweepy.models.ResultSet)

trump_2[0]._json.keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])

In [182]:
#Next step: clean tweets by removing RTs, @_____, and URLs? 

In [191]:
TWEETS_PER_CALL = 200 

In [314]:
def save_tweets(screen_nm, tweets):
    with open(f'C:\\Users\\ryan\\Desktop\\Thinkful DS Sample Data - Main Course\\Unit 4\\Capstone\\{screen_nm}.json', 'w') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)

def get_tweets(screen_nm, desired_ct):
    
    #desired_ct is the number of tweets the user wants to include
    tweet_list = []

    #curr_max_id = math.inf
    #grab the current maximum tweet id for provided screen name, which will be the id of the first tweet stored in the object
    curr_max_id = api.user_timeline(screen_name = screen_nm)[0].id 
    
    #may have to do more than 1 call to the API (if user wants > TWEETS_PER_CALL tweets)
    loops = math.ceil(desired_ct / TWEETS_PER_CALL)
    remaining = desired_ct
        
    for iter in range(loops):
        tweets = api.user_timeline(screen_name = screen_nm, count = min(remaining, TWEETS_PER_CALL), max_id = curr_max_id, include_rts=False)
        
        #save these tweets in a json file for later, in case kernel crashes or anything
        save_tweets(screen_nm, tweets)
        
        #because we are excluding RTs, the len(tweets) will almost always be < desired_ct
        for i in range(min(remaining, TWEETS_PER_CALL, len(tweets))):
            tweet_list.append([tweets[i].text, screen_nm])
                
        remaining = remaining - min(remaining, TWEETS_PER_CALL)
        
        #re-set the maximum id for every TWEETS_PER_CALL tweets, so that we don't include duplicate tweets
        curr_max_id = tweets[-1].id 

    return tweet_list

In [315]:
trump = get_tweets('realDonaldTrump', 2000)

In [320]:
clinton = get_tweets('HillaryClinton', 2000)

In [355]:
#These lists will not actually be 2000 tweets long, due to excluded RT's. Let's use the first 1500 of the final sets

df = pd.DataFrame(trump[0:1500]+clinton[0:1500])

In [356]:
df.columns = ['Tweets', 'Author']

In [367]:
df.head()

,Tweets,Author
0,"The U.S. Coast Guard, FEMA and all Federal and...",realDonaldTrump
1,"Heed the advice of @FLGovScott!\n\n""If you're ...",realDonaldTrump
2,"This is a storm of enormous destructive power,...",realDonaldTrump
3,FLORIDA- \nVisit https://t.co/pdBaD9t8SK to fi...,realDonaldTrump
4,Churches in Texas should be entitled to reimbu...,realDonaldTrump


In [387]:
tweets_doc = nlp(str(df['Tweets']))

In [437]:
def bag_of_words(doc):
    allwords = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.like_url and 
                not token.like_num and not token.is_space]
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(10000)]

In [438]:
bag_of_words(tweets_doc)

['trump',
 'hurricane',
 'be',
 'want',
 'president',
 'donald',
 '’s',
 'republicans',
 'close',
 'campaign',
 'family',
 'hillary',
 'u.s.',
 'coast',
 'guard',
 'tax',
 'hear',
 'irma',
 'speak',
 'world',
 'daca',
 'thank',
 'great',
 'north',
 'dakota',
 'w/',
 'look',
 'go',
 "'s",
 't',
 'stake',
 'refuse',
 'run',
 'america',
 'leave',
 'wrong',
 'way',
 '’re',
 'not',
 'election',
 'people',
 'fema',
 'federal',
 'heed',
 'advice',
 '@flgovscott!\\n\\n"if',
 'storm',
 'enormous',
 'destructive',
 'power',
 'florida-',
 '\\nvisit',
 'fi',
 'churches',
 'texas',
 'entitle',
 'reimbu',
 'message',
 'fellow',
 'americans🇺🇸#irmahurrica',
 'start',
 'reform',
 'cut',
 'allow',
 'pass',
 'gr',
 'sorry',
 'have',
 'abou',
 'incredible',
 'save',
 'tha',
 'epic',
 'proportion',
 'confront',
 'challenge',
 'matter',
 'encourage',
 'path',
 'trip',
 'saudi',
 'arabia',
 'forc',
 'welcome',
 '@whitehouse',
 'amir',
 'sabah',
 'al',
 'ahme',
 'concern',
 'abo',
 'honor',
 'rag',
 'tea',
 '

In [439]:
bow = bag_of_words(tweets_doc)

In [440]:
#How can we clean this up? Need to remove numbers, \n...urls

bow = [re.sub(r'/n','', word) for word in bow]
bow = [re.sub(r'\\n','', word) for word in bow]

In [442]:
len(bow)

206

In [ ]:
#Hmmm... only 206 lemmas kept from 3000 tweets??

In [427]:
#Some of these still don't look quite right, like '@flgovscott!"if'
bow

['u.s.',
 'coast',
 'guard',
 'fema',
 'federal',
 'heed',
 'advice',
 '@flgovscott!"if',
 'be',
 'storm',
 'enormous',
 'destructive',
 'power',
 'florida-',
 'visit',
 'fi',
 'churches',
 'texas',
 'entitle',
 'reimbu',
 'message',
 'fellow',
 'americans🇺🇸#irmahurrica',
 'republicans',
 'start',
 'tax',
 'reform',
 'tax',
 'cut',
 'allow',
 'republicans',
 'pass',
 'gr',
 'republicans',
 'sorry',
 'have',
 'hear',
 'abou',
 'incredible',
 'u.s.',
 'coast',
 'guard',
 'save',
 'tha',
 'hurricane',
 'irma',
 'epic',
 'proportion',
 'confront',
 'challenge',
 'matter',
 'encourage',
 'path',
 'hurricane',
 'trip',
 'saudi',
 'arabia',
 'speak',
 'world',
 'forc',
 'welcome',
 '@whitehouse',
 'amir',
 'sabah',
 'al',
 'ahme',
 'daca',
 'concern',
 'abo',
 'thank',
 'great',
 'honor',
 'hurricane',
 'irma',
 'rag',
 'great',
 'tea',
 'want',
 'company',
 'hire',
 'amp',
 'grow',
 'want',
 'renew',
 'prosperity',
 'restore',
 'op',
 'thank',
 'join',
 'mandan',
 'nd',
 'gov.',
 '@d',
 'won

In [ ]:
##TOPIC Identification via LDA (Latent Dirichlet Allocation)

#https://medium.com/@aneesha/topic-modeling-with-scikit-learn-e80d33668730
#https://www.youtube.com/watch?v=3mHy4OSyRf0

In [428]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

#Any idea on whether this dataset is topic appropriate / large enough to train LDA?

In [429]:
type(documents)

list

In [430]:
documents[0]

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [431]:
len(documents)

11314

Questions:

1. Walkthrough for creating password file and accessing at begining
2. Improving BoW output/fixing partial words


3. Capstone instructions: 

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

        --What is meant by clusters here? 

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

        --Can include features (topics) generated by LSA/LDA which are unsupervised features
        --How would we use an unsupervised model to classify in this case? Would it be possible to use K-Means, for example,
        on the LSA or LDA matrix to cluster?



General question: how would you go about creating a model to optimize something? Like if I wanted to optimize profitability and my goal was to choose the optimal credit line for each customer?

If we have time: Stupid question, but can demo how to run this in another format, like in terminal or spyder?

What are text editors like Atom useful for?